### 0. 引入大模型

In [8]:
from utils import get_qwen_models
llm, chat, embed = get_qwen_models()

### 1. Prompt的技术本质：
- 拼串
- 拼提问的串
- 将提问参数化
- 一切大模型上层开发的根本

### PromptTemplate
- 适合llm大模型
- 最基础也是最重要
- langchain_core

In [1]:
from langchain_core.prompts import PromptTemplate

In [3]:
# 构建一个Pormpt
# Load a prompt template from a template.

prompt = PromptTemplate.from_template(template="请输出{num}种北京美食！")

In [4]:
prompt

PromptTemplate(input_variables=['num'], template='请输出{num}种北京美食！')

In [5]:
prompt.format(num=3)

'请输出3种北京美食！'

In [11]:
result = llm.invoke(input=prompt.format(num=5))
print(result)

好的，北京有许多美味的食物，以下是五种非常有名的北京美食： 

1. 北京烤鸭：一种以脆皮烤鸭为主料的传统名菜，口感鲜美。 

2. 炸酱面：一种以黄豆酱和肉末为调料的面条，味道浓郁可口。

3. 酱爆鸡丁：一种以鸡肉、青椒和豆瓣酱为主要原料的传统名菜，口味鲜美。

4. 豆汁儿：一种以绿豆为原料制成的饮料，口感醇厚。

5. 老北京炸糕：一种以糯米粉和豆沙为原料的传统糕点，外酥里嫩。


In [12]:
prompt.invoke(input=dict(num=4))

StringPromptValue(text='请输出4种北京美食！')

In [13]:
llm.invoke(input=prompt.invoke(input=dict(num=4)))

'北京拥有丰富的美食文化，以下是四种著名的北京美食：\n\n1. **北京烤鸭**：这是北京最著名的菜肴之一。传统的北京烤鸭选用优质肉鸭，经过特殊处理后，在特制的烤炉中烤制而成。外皮酥脆、肉质鲜嫩，通常搭配薄饼、甜面酱和葱丝一起食用。\n\n2. **炸酱面**：这是一道具有浓郁北京风味的传统面食。主要由黄豆酱和肉末制成的炸酱与手擀面条搭配，再加上黄瓜丝、豆芽等蔬菜作为配菜，味道鲜美。\n\n3. **豆汁儿与焦圈**：豆汁儿是一种用绿豆发酵制成的饮品，味道独特，有些人会觉得略带酸味；焦圈则是油炸的环状食品，口感酥脆。两者通常是早餐时的搭配。\n\n4. **卤煮火烧**：这是一种以猪内脏和豆腐为主要原料的传统小吃。将这些材料与特制的卤水一同炖煮，味道浓郁，是许多老北京人喜爱的小吃之一。\n\n每一种都有其独特的风味和制作工艺，值得尝试。'

In [14]:
chain = prompt | llm

In [16]:
chain.invoke(input=dict(num=2))

'北京拥有丰富的美食文化，这里给您介绍两种著名的北京美食：\n\n1. **北京烤鸭**：这是北京最著名的传统菜肴之一。选用优质肉鸭，经过特殊处理和烤制而成，皮脆肉嫩，色泽红润，味道鲜美。通常搭配薄饼、甜面酱、葱丝等一同食用。\n\n2. **炸酱面**：这是一种非常受欢迎的北京传统面食。主要由黄豆酱或甜面酱制成的炸酱与面条搭配，再加上黄瓜丝、豆芽等蔬菜作为配菜，口感丰富，味道独特。\n\n这两种美食不仅在北京非常流行，在中国乃至世界各地都有很高的知名度。'

In [17]:
prompt = PromptTemplate.from_template(template="请列出{num}种{location}美食！")

In [18]:
prompt

PromptTemplate(input_variables=['location', 'num'], template='请列出{num}种{location}美食！')

In [19]:
chain = prompt | llm

In [20]:
chain.invoke(input=dict(num=2, location="广东"))

'广东拥有丰富的美食文化，其中两种著名的美食包括：烧鹅和早茶。烧鹅是一道以烤制鹅肉为主的菜肴，通常搭配甜酱、蒜泥等调料食用，口感香脆可口；而早茶则是一系列点心的总称，其中包括虾饺、糯米鸡、蛋挞等，是广东人早餐或下午茶时常见的美食。'

In [21]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [23]:
output_parser = CommaSeparatedListOutputParser()

In [24]:
output_parser

CommaSeparatedListOutputParser()

In [26]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [44]:
prompt = PromptTemplate.from_template(template="请列出{num}种{location}美食！使用中文输出！\n{output_parser}",
                                     partial_variables={"output_parser": output_parser.get_format_instructions()})

In [45]:
prompt

PromptTemplate(input_variables=['location', 'num'], partial_variables={'output_parser': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'}, template='请列出{num}种{location}美食！使用中文输出！\n{output_parser}')

In [46]:
print(prompt.invoke(input=dict(num=2, location="上海")).text)

请列出2种上海美食！使用中文输出！
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [47]:
chain = prompt | llm

In [54]:
result = chain.invoke(input=dict(num=5, location="山西"))

In [55]:
result

'刀削面, 过油肉, 忻州瓦酥, 临汾牛肉丸子面, 太原酱梅肉'

In [56]:
output_parser.parse(result)

['刀削面', '过油肉', '忻州瓦酥', '临汾牛肉丸子面', '太原酱梅肉']

In [58]:
chain = prompt | llm

In [59]:
chain.invoke(input=dict(num=5, location="山西"))

'刀削面, 过油肉, 忻州瓦酥, 临汾牛肉丸子面, 太原酱梅肉'

In [60]:
prompt = PromptTemplate.from_template(template="请列出{num}种{location}美食！")                                

In [61]:
prompt

PromptTemplate(input_variables=['location', 'num'], template='请列出{num}种{location}美食！')

In [64]:
chain1 = prompt | llm

In [65]:
chain1.invoke(input=dict(num=5, location="山西"))

'好的，这里有五种著名的山西美食：刀削面、烧麦、过油肉、羊杂碎、头脑。刀削面是山西最著名的传统面食之一，口感筋道，汤汁鲜美；烧麦则是以猪肉、虾仁等为馅料，外皮薄而透明，味道鲜美；过油肉是以猪肉为主料，搭配葱、姜、蒜等调料烹制而成，外酥里嫩；羊杂碎则是将羊肉和羊内脏等食材煮熟后切片，再用辣椒油、花椒油等调料拌匀；头脑则是一种由豆腐、木耳、鸡蛋等多种食材组成的热菜，口感丰富。'

In [69]:
chain2 = prompt | llm | output_parser

In [70]:
chain2.invoke(input=dict(num=5, location="山西"))

['好的，这里有五种著名的山西美食：刀削面、烧麦、过油肉、羊杂碎、头脑。刀削面是山西最著名的传统面食之一，口感筋道；烧麦则是用面粉包裹着肉馅蒸制而成，外皮酥脆；过油肉是一道以猪肉为主料的传统菜肴，肉质鲜美；羊杂碎则是一种以羊肉和内脏为原料的炖菜，汤汁浓郁；头脑是一种以豆腐、豆芽等蔬菜为原料的传统炖菜，口感清爽。']

In [72]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [75]:
prompt = PromptTemplate.from_template(template="请列出{num}本值得一读的{type}书！\n{format}",
                                     partial_variables={"format": output_parser.get_format_instructions()})

In [76]:
prompt

PromptTemplate(input_variables=['num', 'type'], partial_variables={'format': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'}, template='请列出{num}本值得一读的{type}书！\n{format}')

In [85]:
prompt = PromptTemplate.from_template(template="请列出{num}本值得一读的{type}书！\n你的返回应当是用逗号分开的一系列的值，比如： `苹果, 桃, 梨` 或者 `苹果,桃,梨`")

In [86]:
prompt

PromptTemplate(input_variables=['num', 'type'], template='请列出{num}本值得一读的{type}书！\n你的返回应当是用逗号分开的一系列的值，比如： `苹果, 桃, 梨` 或者 `苹果,桃,梨`')

In [87]:
chain = prompt | llm

In [92]:
result = chain.invoke(input={"num":2, "type":"爱情"})

In [93]:
output_parser.parse(result)

['梦里花落知多少', '一瞬的永恒']

In [95]:
def get_result(num, category):
    question = f"请列出{num}种值得一读的{category}书。\n你的返回应当是用逗号分开的一系列的值，比如： `苹果, 桃, 梨` 或者 `苹果,桃,梨`"
    result = llm.invoke(input=question)
    return result

In [96]:
get_result(num=3, category="数学")

'斐波那契数列,黎曼猜想,数学之美'

In [99]:
prompt = PromptTemplate(template="请讲{num}个关于{item}的冷笑话！")

In [100]:
prompt

PromptTemplate(input_variables=['item', 'num'], template='请讲{num}个关于{item}的冷笑话！')

In [102]:
prompt1 = prompt.partial(item="汽车")

In [103]:
prompt1.invoke(input={"num":3})

StringPromptValue(text='请讲3个关于汽车的冷笑话！')

In [104]:
llm.invoke(prompt1.invoke(input={"num":3}))

'当然可以，接下来是三个关于汽车的冷笑话：\n\n1. 为什么汽车经常去参加同学聚会？\n   因为它想展示一下自己的马力。\n\n2. 你知道汽车最怕什么科目吗？\n   是地理课，因为它们总是担心会迷路。\n\n3. 为什么汽车不喜欢吃快餐？\n   因为它们害怕变成“油”腻的家伙。\n\n希望这些笑话能给你带来一丝欢笑！'

### FewShotPromptTemplate
- 少样本学习
- 大模型"能力涌现"的典型代表
- 大模型本身没有在这个方向上做大量的训练，但是，可以通过我给定的几个少数的例子，学到背后的知识，从而快速处理相关的问题
- 你给大模型打个样，然后大模型就可以照猫画虎来解决相关的问题！！！

In [153]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate

In [154]:
# 首先，构建一个样例的模板
example_prompt = PromptTemplate.from_template(template="输入：{in}\n输出：{out}")

In [155]:
example_prompt

PromptTemplate(input_variables=['in', 'out'], template='输入：{in}\n输出：{out}')

In [156]:
examples = [
    {"in": "深圳", "out":"华南"},
    {"in": "阳泉", "out":"华北"},
    {"in": "锦州", "out":"东北"},
    {"in": "酒泉", "out":"西北"}
]

In [157]:
prompt = FewShotPromptTemplate(example_prompt=example_prompt, 
                               examples=examples,
                               prefix='请学习我给定的样例，并据此回答我提出的问题：\n"""',
                               suffix='"""\n输入：{input}\n输出：')

In [161]:
# result= prompt.invoke(input={"input": "大庆"})

In [165]:
print(result.text)

请学习我给定的样例，并据此回答我提出的问题：
"""

输入：深圳
输出：华南

输入：阳泉
输出：华北

输入：锦州
输出：东北

输入：酒泉
输出：西北

"""
输入：大庆
输出：


In [163]:
chain = prompt | llm

In [164]:
chain.invoke(input={"input": "绍兴"})

'华东\n\n根据提供的样例信息，可以推断这里指的是中国的地理区域。"绍兴"位于中国的东部地区，因此归类于“华东”。'

In [167]:
llm.invoke(result.text)

'东北\n\n根据给定的样例，我们可以看出输入的地名对应着中国的不同地理区域。大庆位于中国东北地区，因此输出应为“东北”。'

### PipelinePromptTemplate

In [168]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PipelinePromptTemplate

In [169]:
full_template = '''
{expect}
{example}
{question}
'''

In [170]:
full_prompt = PromptTemplate.from_template(template=full_template)

In [171]:
full_prompt

PromptTemplate(input_variables=['example', 'expect', 'question'], template='\n{expect}\n{example}\n{question}\n')

In [177]:
# prefix
expect_prompt = PromptTemplate.from_template(template='请学习我给定的样例，并据此回答我提出的问题：\n"""')

In [178]:
example_prompt = PromptTemplate.from_template(template="""
输入：深圳
输出：华南

输入：阳泉
输出：华北

输入：锦州
输出：东北

输入：酒泉
输出：西北
""")

In [190]:
question_prompt = PromptTemplate.from_template(template='"""\n输入：{in}\n输出：')

In [191]:
# List[Tuple[str, langchain_core.prompts.base.BasePromptTemplate]]
pipeline_prompts = [("expect", expect_prompt), 
                    ("example", example_prompt),
                    ("question", question_prompt)]

In [192]:
prompt = PipelinePromptTemplate(final_prompt=full_prompt,
                      pipeline_prompts=pipeline_prompts)

In [193]:
prompt

PipelinePromptTemplate(input_variables=['in'], final_prompt=PromptTemplate(input_variables=['example', 'expect', 'question'], template='\n{expect}\n{example}\n{question}\n'), pipeline_prompts=[('expect', PromptTemplate(input_variables=[], template='请学习我给定的样例，并据此回答我提出的问题：\n"""')), ('example', PromptTemplate(input_variables=[], template='\n输入：深圳\n输出：华南\n\n输入：阳泉\n输出：华北\n\n输入：锦州\n输出：东北\n\n输入：酒泉\n输出：西北\n')), ('question', PromptTemplate(input_variables=['in'], template='"""\n输入：{in}\n输出：'))])

In [194]:
print(prompt.invoke(input={"in":"大理"}).text)


请学习我给定的样例，并据此回答我提出的问题：
"""

输入：深圳
输出：华南

输入：阳泉
输出：华北

输入：锦州
输出：东北

输入：酒泉
输出：西北

"""
输入：大理
输出：



In [197]:
llm.invoke(prompt.invoke(input={"in":"三亚"}).text)

'华南\n\n根据提供的样例，可以推断出这些输入的地名与其所在的中国大地理区域相对应。"三亚"位于中国的海南省，因此它属于“华南”地区。'